# Disclaimer

NOTE: This notebook is not an endorsement of gambling of any nature and is purely for intellectual curiousity.

# Introduction

Hello there fellow sports fans! Almost every basketball fan has once tried to predict the outcome of a playoff series, and in this notebook I will attempt to do the same. Before the start of every series Las Vegas releases betting odds of who they think is the favorite and by how much. (Click here to learn about American betting odds work https://www.vegasinsider.com/how-to-bet-sports/odds/#us). I will retroactively analyze the all the playoff series since 1969 to answer questions such as: 1.) How often did the favorite actually win the series? 2.) How often did the team with homecourt advantage win the series? 3.) What seed were the NBA champions historically?

# Web Scraping

Data will be sourced from https://www.sportsoddshistory.com/ and converted to .csv format

In [ ]:
import requests

URL = "https://www.sportsoddshistory.com/nba-playoffs-series/?y=all&o=s"
page = requests.get(URL)

print(page.text)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

all_playoff_series = soup.find_all("tr")

In [ ]:
for series in all_playoff_series:
    print(series, end="\n"*2)


In [ ]:
# Remove first and last elements of list since these do not represent playoff data

all_playoff_series = all_playoff_series[1:-1]

In [ ]:
for series in all_playoff_series:
    print(series, end="\n"*2)

In [ ]:
import re

pattern = re.compile(r'(.*) \((\d)\)')

# Take each table row and convert data to csv format
def row_to_string(row):
    td_elements = row.find_all('td')
    return_str = ''
    for td in td_elements:
        text = td.text.strip()
        
        # Split 'Team (Seed#)' into 'Team,Seed#'
        if (re.fullmatch(pattern, text)):
            team_seed_str = re.split(pattern, text)
            return_str += team_seed_str[1] + ',' + team_seed_str[2] + ','
        else:
            return_str += text + ','
    
    # Remove trailing comma
    return_str = return_str[:-1]
    return return_str

In [ ]:
row_to_string(all_playoff_series[50])

In [ ]:
csv_rows = []
for series in all_playoff_series:
    csv_rows.append(row_to_string(series))

In [ ]:
csv_rows

In [ ]:
csv_header = 'Year,Conference,Round,Team1,Seed1,Odds1,Team2,Seed2,Odds2,WinningTeam,Result'

In [ ]:
import os

if not(os.path.exists('playoffs.csv')):
    f = open('playoffs.csv', 'w')
    f.write(csv_header + '\n')
    for row in csv_rows:
        f.write(row + '\n')
    f.close()